In [2]:
import cv2
import torch
from ultralytics import YOLO
from collections import defaultdict

# ==================== 配置参数 ====================
VIDEO_PATH = "highway_d.avi"          # 输入视频路径
OUTPUT_PATH = "gpu_latest_output.mp4"  # 输出视频路径
COUNT_LINE_Y = 700                  # 计数线位置
CLASS_NAMES = [2, 5, 7]             # COCO车辆类别ID
TRACKER_CONFIG = "botsort.yaml"     # 跟踪算法
DEVICE = "cuda"                     # 自动选择可用GPU（多卡可用 "cuda:0"）

# ==================== GPU加速核心逻辑 ====================
def main():
    # 检查CUDA可用性
    if not torch.cuda.is_available():
        raise RuntimeError("CUDlA不可用，请检查驱动和PyTorch安装！")
    
    # 加载最新YOLOv8模型并移至GPU
    model = YOLO("yolov8l.pt")
    model.to(DEVICE)  # 最新版本支持直接.to(device)
    
    # 视频流配置
    cap = cv2.VideoCapture(VIDEO_PATH)
    assert cap.isOpened(), "视频打开失败"
    fps = cap.get(cv2.CAP_PROP_FPS)
    width, height = int(cap.get(3)), int(cap.get(4))
    writer = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    
    # 跟踪与计数状态
    track_history = defaultdict(list)
    counted_ids = set()
    total_count = 0

    # 实时处理循环
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break

        # ========== YOLOv8 GPU加速推理 ==========
        results = model.track(
            frame, 
            persist=True,               # 跨帧持续跟踪
            tracker=TRACKER_CONFIG,     # 指定跟踪算法
            classes=CLASS_NAMES,        # 过滤车辆类别
            device=DEVICE,              # 显式指定GPU设备
            verbose=False,              # 关闭控制台输出
            imgsz=640                   # 控制推理分辨率（平衡速度与精度）
        )

        # ========== 后处理与可视化 ==========
        if results[0].boxes.id is not None:
            # 提取检测结果（自动处理设备转换）
            boxes = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.cpu().numpy().astype(int)
            
            # 遍历每个检测目标
            for box, track_id in zip(boxes, track_ids):
                x1, y1, x2, y2 = box
                center_x, center_y = int((x1 + x2) / 2), int((y1 + y2) / 2)

                # 轨迹记录（GPU无关，纯CPU操作）
                track = track_history[track_id]
                track.append((center_x, center_y))
                if len(track) > 30:
                    track.pop(0)

                # 计数逻辑
                if len(track) >= 2 and track[-2][1] < COUNT_LINE_Y and center_y >= COUNT_LINE_Y:
                    if track_id not in counted_ids:
                        total_count += 1
                        counted_ids.add(track_id)

                # 绘制检测框与ID（OpenCV操作在CPU）
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1)-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # 绘制计数线与统计信息
        cv2.line(frame, (0, COUNT_LINE_Y), (width, COUNT_LINE_Y), (0, 0, 255), 3)
        cv2.putText(frame, f"Total Vehicles: {total_count}", (20, 50), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

        # 输出视频写入
        writer.write(frame)
        
        # 实时预览（ESC退出）
        cv2.imshow("YOLOv8 GPU Counting", frame)
        if cv2.waitKey(1) == 27:
            break

    # 释放资源
    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    print(f"处理完成！总车辆数: {total_count}")

if __name__ == "__main__":
    main()

处理完成！总车辆数: 46
